# Week 8 Lecture Notes:
# Network Programming Using Sockets (2/27/25) 
### Covering: Overview of Computer Networking, deeper dive into the TCP/IP and UDP/IP Protocols
### Programming TCP & UDP using sockets
- need to use buffers for assignment (make sure to zero them out at the start)
- start by implementing demo code into the assignment that help with degugging(print messages) to watch control flow & communication between sever and client on the local 127.0.0.1 -> then implement other things from previous assignments.
- dsh_cli is fully set up - setting up defaults
    - for server use 0.0.0.0:whateverport
    - for client use 127.0.0.1:same port
- need to write code for exec_local_cmd_loop() and exec_remote_cmd_loop() in dshlib.c & rsh_cli.c
    - for exec_local_cmd_loop() simply use the code from last weeks assignment
    - need to implement rsh_server.c
- make sure to use end of string character to signal end of message "0x04"

- Network protocols are a set of rules that govern how data is formatted, sent, and recieved between devices on a computer network.
    - the most widely used network protocol is TCP/IP
- Network Protocols can be stateful or stateless
    - Statless: each conncection between sender and reciever is only alive to send that one message and receive a response => for each exhange a new connection is made
        - stateless protocols will be faster, will have less overhead but will not be fully reliable
    - Statefull protocol maintains a connection between a sender and a reciever for the duration of a process and allowing both the receiver and sender to **maintain a protocol "state" to keep track of an exchange "context"**
        - statefull protocols will have more robust capabilities such as reliability but will be slower with more overhead
    - most application protcols will be statefull whereas most lower-level network protocols tend to be stateless(with the exception of TCP)
- OSI model(network model) - abstract way to define a network stack in 7 layers
    - layers: 1 physical(computer network card)-> 2 Data Link -> 3 Network(Ip addresses) -> 4 Transport -> 5 Session -> 6 Presentation -> 7 Application
    - if not concerned with physical components, layers 1-4 can all be considered transport
    - layers 5-7 can be thought of as the DATA which is a combination of segmenets made up of data packets.
    - layers 5/6 are very abstract no strong purpose.
    - segments(in transport layer) have packets, packets have frames, frames have bits
- TCP/IP Network Model(widely used) - concrete implementation of a network stack
    - 5 layers: 1 physical(computer network card)-> 2 Data Link -> 3 Network(Ip addresses) -> 4 Transport -> 7 Application(5,6 contents covered in this layer (application layer))
    - TCP is "FULL-Duplex" meaning through one TCP connection you can send and recieve messages at the same time (2 threads)
    - TCP is STREAM protocol(a stream of bytes flow) -NOT a individual messages(not message based)
        - this means you need to build into your application where messages begin and end (define a endpoint of the stream to the reciever."EOF")
- UDP is a message based protocol(2nd most popular approach) individual messages are packaged and delivered - Not very reliable given that messages can get lost and you would never know that you missed a message (not guarunteed to be recieved).
- getting data across a network:
    - establish a transport link between browser and an application(transport layer #4)(holds level 7 traffic) (uses Port numbers to locate the application  on individual nodes)
    - establish a network link between source node on your computer to a server node supporting the application(layer 3)(IP Adresses - locating Nodes on a broader network)
    - establish frame links that move data from the source node to a router then to the cloud and then to another router and then to the pyhsical server node supporting the application.(layer1/layer2)(MAC Addresses - traversing next "Hop")
- to get data across a network you need a destination address and a senders address(level 4) lower levels(1-3) are concerned with how it gets to that destination address.
    - key addresses:
        1- mac address -  unique machine identifier used to send mail(data) hop-to-hop from sender to recivever (intermediate destinations(routers))
        2- IP address - 32 bit number to represent a device on a particular network (an represent an API endpoint). ex.10.50.47.88
        3- port address - 32 bit number used to represent an application on a specific device ex. Port 443(https default port) & Port 80(http default port)
- Sockets
    - **Sockets are the programming interface provided by the operating system enabling applications to write network programs**
        - syscalls are made to sockets to utilize a network connection.
    - Sockets sit inbetween application layer(7)(what users interface with) and trasnport/network/datalink layers(4-2 - implemented by the operating system) in the network Stack
    - Sockets provide a "file" abstraction over making network connections
    - socket syscall in C: - creating a TCP socket in C
        - listen_socket = socket(AF_INET, SOCK_STREAM, 0)
        - where AF_INET is the domain - IPv4 vs. IPv6
        - where SOCK_STREAM - is the type, streaming protocol or data varing(UDP)
        - where 0 is the protocol - 0 is the default TCP protocol
        - acts as a file descriptor (integer_id of the file descriptor)
    - "setsockopt()" is a method used to set up socket options for a TCP socket
    - when binding a server to an address that means we are "listening" on it
        - components of an address: family, address, port
            - ex. addr.sin_family = AF_INET
                  addr.sin_addr.s_addr = htonl(INADDR_ANY)
                  addr.sin_port = htons(PORT_NUM)
        - 10.10.10.10:1234 is (IPV4):(port) is a representation as "address"
        - "htonl()" is a method host to network long will take into consideration your endianess and architecture of your system to make sure it configures the address correctly
    - listening 0.0.0.0 means listen on any available network interfaces
    - need to use "bind()" to bind a client to a server and their can only be one server connected to a client at a time
        - need to re-use this boiler plate code for assignment.
    - when you activate it by listen() you can use process_requests()(the functionality of other assignments.) and then close() the TCP socket
    - using the "accept()" method on a socket will lock it for use
    - use "recv()" to get the data of the message